<a href="https://colab.research.google.com/github/NatumanyaDuncan/secone_one-/blob/main/Yelp_API_Package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API Calls Using the YelpAPI Package
Thankfully, we do not need to construct our API calls manually. There is a Python package for most APIs that we can leverage. For more information on how to construct API calls manually, see the optional "Manually Constructing API Calls " lesson at the end of this week.

For Yelp, we have the YelpAPI Package ([Documentation](https://github.com/gfairchild/yelpapi))

<img src="https://assets.codingdojo.com/boomyeah2015/codingdojo/curriculum/content/chapter/1648664855__yelpapi-package.png">

#### How to Use YelpAPI:
* To use the package, we must create an instance of the YelpAPI class.
  * We will then use this variable to execute all of our queries.  

First, be sure to open your local json file with your Yelp API credentials. You should have this saved in a secret folder in a json file. If not, please refer to the "Saving and Using API Credentials Lesson." Be sure to use YOUR path. It should look similar to the path used below.

In [ ]:
import json
with open('/Users/YOUR_PATH.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

In [ ]:
# import the YelpAPI Class
from yelpapi import YelpAPI
# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

* To use the "businesses search" endpoint,  we will use the yelp_api.search_query method.
 *  If we inspect the docstring for the function (either run the help function on it or place your cursor inside the parenthesis for it and hit Shift+Tab), we see that it doesn't tell us very much.

In [ ]:
help(yelp_api.search_query)

* The only arguments/parameters that the docstring mentions are location or latitude & longitude.
* Notice that the link to the official API documentation is linked instead.
  * This is because this documentation only shows what the required parameters are, but it will accept ANY of the parameters that are accepted by the business_search endpoint.

In [ ]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location='NY, NY',
                                       term='Pizza')
print(type(search_results))
search_results.keys()

* The package returns the results in the JSON format we have been exploring. Note that the exact results may vary as the Yelp site is constantly changing.

In [ ]:
search_results['total']

In [ ]:
biz = pd.DataFrame(search_results['businesses'])
biz.head(2)

(Your results may vary from what is shown, but should generally look like the output above)
Pagination and "offset"
Pagination can be understood through this brief demonstration:


**Total Results vs. Businesses**

* If we check the total number of results, we can see that we had 12,000 businesses that met our search criteria.

In [ ]:
## total number of matching businesses
search_results['total']

* However, if we check our list of businesses, we will see that we only received data for 20 businesses.

In [ ]:
## how many businesses in our results
len(search_results['businesses'])

We have 12000 results, but the length is only 2.? What's going on?

If you think about the results of a Google search, we usually have MANY more results that Google will display at one time. When we scroll down to the bottom of the results, we can see that Google has divided the results into several PAGES of results instead of one extremely long page.

<img src="https://assets.codingdojo.com/boomyeah2015/codingdojo/curriculum/content/chapter/1648664842__google_results_pages.png">  

This is what Yelp Fusion is doing as well! The general term for this is "Pagination".



The Yelp API will only return a "page" of 20 results at a time.

* If we want to get the next page of results, we will perform another API call, but we will add an additional argument called "offset."
  * The offset is what # result to use as the FIRST result for the page.
  * If we had 20 businesses in our first result, we would want to add an offset of 20.

In [ ]:
# add offset to our original api call
search_results = yelp_api.search_query(location='NY, NY',
                                       term='Pizza',
                                       offset = 20)


In [ ]:
biz20 = pd.DataFrame(search_results['businesses'])
biz20.head(2)

You should have different results than your original call when you make the offset call.

Now you can combine the results (so far) into one dataframe using pd.concat()

In [ ]:
## concatenate the previous results and new results.
businesses = pd.concat([biz, biz20],
                      ignore_index=True)
display(businesses.head(3), businesses.tail(3))

# [Concepts of Efficient API Extraction](https://login.codingdojo.com/m/376/12529/88076)

# Code for Efficient API Extraction

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time

**Define Search**

To allow us to easily perform different searches in the future, we will define variables for LOCATION and TERM set for our particular search conditions. Then, when we want to use a different location or term, we can just redefine these variables. This streamlines the code and makes it more readable and reproducible.

In [ ]:
# set our API call parameters
LOCATION = 'NY,NY'
TERM = 'Pizza'

Create a results-in-progress JSON file, but only if it doesn't exist.

This is the file where your results will be saved. Note: you must rename your JSON_FILE for different queries to prevent confusing results from other searches. We recommend you include your search terms in the filename.

In [ ]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

Check if our JSON_FILE already exists. This will prevent us from accidentally overwriting an existing file.

If it doesn't exist:

* Create any folders needed for the file path.
* Save an empty list as JSON_File

In [ ]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist:
if file_exists == False:

    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)


    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')


    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to a new file.

OR

[i] Data/results_in_progress_NY_pizza.json already exists.

If you get the "already exists" message, you already have a file of this name. If you have made this file and have already started making these API calls, you can continue to use this file. If not, you would need to change the name of the file to start with an empty file.


**Determine how many results are already in the file**

Load the results file to determine the # of results we have previously retrieved. If you just created the file, you would expect it to be empty.

We will use this as our offset parameter for our API call. Even if this is your first API call, and the number is 0, we want to define "n_results" based on the length of "previous_results."

In [ ]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)

## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


**Figure out how many pages of results we will need**

We will perform our first query to get our first page of results and the total number of results. We will then (via code) calculate how many pages we will need to retrieve all of our results.

In [ ]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

In [ ]:
## How many results total?
total_results = results['total']
total_results

In [ ]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

There are over 11000 businesses to retrieve from our API, and we can get 20 results at a time (per "page").
* We can calculate the # of results remaining by subtracting our offset (length of our previous results) from our total.
* Then we can determine how many pages we will need by dividing the results by 20 (or whatever the value happens to be for results per page)
* Note that we need to **round u**p the number of pages in order to get all of the results. Even if there is only 1 result on the last page, we want to include that page! To do this, we will use math.ceil.

In [ ]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

When this example was written, there were 11200 results and 20 results per page. 11200 /20 = 560. But what if we had 11210 results? Rounding UP would give us 561 pages. We expect the first 560 pages to have 20 results and the last page to have the final 10 results. Notice that we have assigned the number of pages as n_pages here. We will use this value in our next segment of code.

You can see that having to manually go through 560 pages would be quite time-consuming and inefficient! First, we are going to save the first page into our file, and then we will add to it with a for loop.



**Add this page of results to .json file**

Our API returns our results in JSON format, with the businesses in a list of dictionaries. We will append the first page of businesses to our previous_results (which is very likely empty) and then save it to disk.

In [ ]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

**Set up a progress bar in our for loop.**

To keep us informed about where we are in our loop, we will add a progress bar to our for loop.

TQDM is a package designed for adding animated progress bars to Python processes. **It is not currently included in your dojo-env, so you are going to install it manually** by opening a new Terminal/GitBash window and running the following command:  
`pip install tqdm`

TQDM is easy to use and simply needs to know what we are looping through. If you wanted to test tqdm in action, but your loop is too fast, you can import time and use time.sleep to add a pause within your for loop. We will also use time.sleep when executing many API calls so that we do not overwhelm the server.

In [ ]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2)

**For Loop to call each page**  

The loop below will iterate through each page of the results by starting at the appropriate offset. It will then append the results to the previous_results. This may take some time, so check out the progress bar!

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):

    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM,
                                    offset=n_results)

    ## append new results and save to file
    previous_results.extend(results['businesses'])

    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

    # add a 200ms pause
    time.sleep(.2)

The code above will return an error. To understand the origin of this error, let's examine the bottom of our error message,

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

It is telling us that we asked for too many results and that we can only get <= 1,000 results.

This is the limitation of using the free tier of Yelp's API. If we were to pay a monthly fee for better access, we would not hit this limitation. Unfortunately, there is no way to adjust our calls to skip those first 1,000. So we can only ever get the same first 1,000 results.

So what can we do about it now so that we run our code without error?    

## Handling queries with >1000 Results
To get around this error, we can add an extra logic check to see if the length of movies we have so far (n_results) + the # of results on each page (results_per_page) is greater than 1,000.

If it is greater than 1,000, we will use a break to end our loop early.  

**Deleting Our Previous Results FIle**
* Let's give ourselves a fresh start with our new and improved loop. Let's delete our previous results file.
We could accomplish this manually or programmatically.  

**The Manual Way**
* We could do this manually by using the Files page in Jupyter to find the file in our results folder and delete it. To delete a file with Jupyter check the check box next to the file you want to delete.

* You should now see additional buttons appear above the list of files.

  * Click on the red trash can icon to delete the file.

  <img src="https://assets.codingdojo.com/boomyeah2015/codingdojo/curriculum/content/chapter/1662056603__delete%20with%20jupyter.png">

**The Programmatic Way**  

Just like we used the os module to create new folders, we can also use it to delete (or remove) files.

We simply use the os.remove function and pass in our filename to delete.

We can then use our os.path.isfile function again to confirm the file no longer exists.

In [ ]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

* Now that we've deleted the file, we need to re-run our code to create it.

* This process is begging to be turned into a function so we can easily repeat it.

* While we are making it a function, we will add the option to delete the JSON file if it already exists, just like we did above.

  * So let's make a create_json_file function that accepts the JSON_FILE filename as the first argument and a second argument called delete_if_exists and set it to False by default.

  * This way, it will not automatically delete previous search results. We will have to explicitly say delete_if_exists = True to do so.

In [ ]:
def create_json_file(JSON_FILE,  delete_if_exists=False):

    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    ## If it DOES exist:
    if file_exists == True:

        ## Check if user wants to delete if exists
        if delete_if_exists==True:

            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")


    ## If it does NOT exist:
    else:

        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

Now that we have our new function, we can use it with delete_if_exists=True to delete our previous results and start over. We will also need to repeat the steps to recreate our n-results, total_results, results_per_page, and n_pages variables that we created before our first attempted loop.

In [ ]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)

## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):

    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)

    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break

    ## use n_results as the OFFSET
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM,
                                    offset=n_results)



    ## append new results and save to file
    previous_results.extend(results['businesses'])

    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

    time.sleep(.2)

### After the loop has finished

**Convert .json to dataframe**  
Load in the "results in progress" JSON file into a DataFrame:

In [ ]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

**Check for and remove any duplicate results.**

In [ ]:
# check for duplicate results
final_df.duplicated().sum()

* Because our Yelp results include columns that contain lists, we cannot check every column in the dataframe for duplicates.
* Instead, we can use the subset argument for df.duplicated() and df.drop_duplicates() to only check the id column for duplicates.

In [ ]:
# check for duplicate ID's
final_df.duplicated(subset='id').sum()

In [ ]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()